In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.6.0


In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 4s 1us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-9-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [10]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [11]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 05m 39s]
val_accuracy: 0.9023333191871643

Best val_accuracy So Far: 0.9120000004768372
Total elapsed time: 00h 46m 20s
INFO:tensorflow:Oracle triggered exit


In [12]:
model=tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        115248    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1486912   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,603,770
Trainable params: 1,603,770
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 341s 202ms/step - loss: 0.1473 - accuracy: 0.9458 - val_loss: 0.2603 - val_accuracy: 0.9088
Epoch 5/10
1688/1688 [==============================] - 358s 212ms/step - loss: 0.1094 - accuracy: 0.9593 - val_loss: 0.2660 - val_accuracy: 0.9113
Epoch 6/10
1688/1688 [==============================] - 360s 214ms/step - loss: 0.0784 - accuracy: 0.9709 - val_loss: 0.2971 - val_accuracy: 0.9098
Epoch 7/10
1688/1688 [==============================] - 340s 202ms/step - loss: 0.0605 - accuracy: 0.9777 - val_loss: 0.3619 - val_accuracy: 0.9115
Epoch 8/10
1688/1688 [==============================] - 372s 221ms/step - loss: 0.0455 - accuracy: 0.9835 - val_loss: 0.4132 - val_accuracy: 0.8987
Epoch 9/10
1688/1688 [==============================] - 399s 236ms/step - loss: 0.0353 - accuracy: 0.9864 - val_loss: 0.4775 - val_accuracy: 0.9100
Epoch 10/10
1688/1688 [==============================] - 1804s 1s/step - loss: 0.0279 - accuracy: 0.9901 - val_l